# Chapter 3 Solving Non-invertible Linear System

Let us solve $A x = b$ when $A$ is non-square or not of full rank. We will use $LU$-decomposition.

In [21]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [23]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [149]:
def solve_Ax_b(A, b):
    m = A.shape[0]
    n = A.shape[1]
    Aaug = np.hstack((A,-b))
    # LU-decomposition
    P, L, U = sp.linalg.lu(Aaug)
    print(U)

    # find pivot entries
    epsilon = 10**(-10)     # Setting epsilon for negligibles
    pivot = n+1
    rank_u = 0
    pivots = []
    while pivot >= 0:
        non_zero_indices = np.nonzero(U[rank_u,:]**2 > epsilon)
        if np.size(non_zero_indices) == 0 or pivot == non_zero_indices[0][0]:
            pivot = -1
        else:
            pivot = non_zero_indices[0][0]
            pivots.append(pivot)
            rank_u += 1

    print('Rank = ')
    print(rank_u)
    print('Pivot variables =')
    print(np.int64(pivots))

    if n in pivots:
        print("The linear system is infeasible since right hand b increases the rank.")
        return
    
    # find column indices of free variables
    free_variables = np.sort(list(set(range(Aaug.shape[1]))- set(pivots)))
    print("Free variables =")
    print(free_variables)

    # Collect pivot columns and free variables
    U_pivot = U[:rank_u, pivots]
    print('pivot columns of U = ')
    print(U_pivot)
    U_free = U[:rank_u, free_variables]
    print('free variable columns of U = ')
    print(U_free)

    # Solve for the pivot part of a basis of the null space of A
    X_pivot = np.linalg.solve(U_pivot,-U_free)
    print('pivot part of X = ')
    print(X_pivot)

    # remove the right hand b from free variables
    free_variables = free_variables[:-1]

    # Combine pivot part and free variable part
    X = np.hstack((np.eye(n)[:,free_variables], np.zeros(n).reshape(-1,1)))
    X[pivots,:] = X_pivot
    print('X = ')
    print(X)

    print(np.allclose(A @ X, np.hstack((np.zeros((m,n-rank_u)),b))))    

In [160]:
# Setting solvability
solvable = 1
# Setting parameters
m=10
n=11
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)
if solvable:
    b = A @ np.random.randn(n,1)
else:
    b = np.random.randn(m,1)
    
solve_Ax_b(A,b)

[[-5.208  -1.7143 -1.3894 -3.419  -0.4245  2.5248  0.0874 -3.7586 -0.6346 -3.308  -2.5765 -9.6977]
 [ 0.     -6.1267 -3.4656  1.1619 -1.2636  2.0176  0.1154 -2.5634 -1.9196 -1.7937 -5.0378 -3.354 ]
 [ 0.      0.      6.4203  1.1635  2.6316 -3.0769 -4.4004  3.6119 -0.4827  2.1537  7.9478  5.9465]
 [ 0.      0.      0.      2.9756  2.3883  2.4271  3.0355 -2.0229 -0.8887 -0.4646  0.1042 -6.4807]
 [ 0.      0.      0.      0.     -1.1239 -1.6551  1.6421 -0.8081 -1.3801 -0.141   0.0658 -0.2473]
 [ 0.      0.      0.      0.      0.      1.7144  1.3445 -4.0956  0.3604 -2.5856 -1.4832 -4.5061]
 [ 0.      0.      0.      0.      0.      0.     -3.5075  6.631   1.575   5.9563  4.7265  9.3414]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.      0.      0.    ]]
Rank = 
7